In [49]:
from GNLP.corpus import Corpus

2020-10-20 23:59:22,355	INFO -- MainProcess corpus.py:17 -- Welcome to the Georgian NLP toolset demo


In [50]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [52]:
corp = Corpus()
corp.from_file('data/corpuses/kawiki-latest-pages-articles_preprocessed.txt')

{   'counted': 0,
    'prepro_sequence': 0,
    'sequence': 0,
    'stop_words': 373,
    'tokens': 0}

In [53]:
corp.file2sequence(preprocess=True)
corp.count_tokens()

2020-10-21 00:04:29,509	INFO -- MainProcess corpus.py:113 -- ვიწყებ data/corpuses/kawiki-latest-pages-articles_preprocessed.txt-ის დამუშავებას
2020-10-21 00:04:29,602	INFO -- MainProcess corpus.py:126 -- წინადადების რაოდენობა: 74233


In [56]:
corp.count_tokens()

In [60]:
len(corp.counted)

120830

In [59]:
corp.correction("თბილასი")

'თბილისი'

In [5]:
#!wget https://storage.googleapis.com/bert_models/2018_11_23/multi_cased_L-12_H-768_A-12.zip
#!unzip GNLP/data/bert_model/multi_cased_L-12_H-768_A-12.zip

Archive:  GNLP/data/bert_model/multi_cased_L-12_H-768_A-12.zip
   creating: multi_cased_L-12_H-768_A-12/
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.meta  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.data-00000-of-00001  
  inflating: multi_cased_L-12_H-768_A-12/vocab.txt  
  inflating: multi_cased_L-12_H-768_A-12/bert_model.ckpt.index  
  inflating: multi_cased_L-12_H-768_A-12/bert_config.json  


In [40]:
BERT_VOCAB = 'multi_cased_L-12_H-768_A-12/vocab.txt'
BERT_INIT_CHKPNT = 'multi_cased_L-12_H-768_A-12/bert_model.ckpt'
BERT_CONFIG = 'multi_cased_L-12_H-768_A-12/bert_config.json'

In [6]:
#!pip install bert-tensorflow 
# conda install tensorflow = 1.14
#!pip install numpy

  Using cached bert_tensorflow-1.0.4-py2.py3-none-any.whl (64 kB)
     |████████████████████████████████| 320.4 MB 23.6 MB/s eta 0:00:01
     |████████████████████████████████| 20.1 MB 42.2 MB/s eta 0:00:01
     |████████████████████████████████| 3.8 MB 37.9 MB/s eta 0:00:01
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
     |████████████████████████████████| 1.3 MB 44.0 MB/s eta 0:00:01
  Using cached tensorflow_estimator-2.3.0-py2.py3-none-any.whl (459 kB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached tensorboard-2.3.0-py3-none-any.whl (6.8 MB)
  Using cached termcolor-1.1.0.tar.gz (3.9 kB)
     |████████████████████████████████| 2.9 MB 38.8 MB/s eta 0:00:01
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached absl_py-0.10.0-py3-none-any.whl (127 kB)
  Using cached gast-0.3.3-py2.py3-none-any.whl (9.7 kB)
     |████████████████████████████████| 114 kB 37

In [31]:
!wget https://raw.githubusercontent.com/google-research/bert/master/modeling.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/optimization.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/run_classifier.py 
!wget https://raw.githubusercontent.com/google-research/bert/master/tokenization.py 

--2020-10-20 23:38:53--  https://raw.githubusercontent.com/google-research/bert/master/modeling.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 37922 (37K) [text/plain]
Saving to: ‘modeling.py’

modeling.py         100%[===================>]  37.03K  --.-KB/s    in 0.06s   

2020-10-20 23:38:54 (601 KB/s) - ‘modeling.py’ saved [37922/37922]

--2020-10-20 23:38:54--  https://raw.githubusercontent.com/google-research/bert/master/optimization.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.192.133, 151.101.128.133, 151.101.64.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.192.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6258 (6.1K) [text/plain]
Saving to: ‘opt

In [36]:
import modeling
import optimization
import run_classifier
import tokenization
import tensorflow as tf
import numpy as np


In [37]:
import unicodedata

def whitespace_tokenize(text):
    text = text.strip()
    if not text:
        return []
    tokens = text.split()
    return tokens

class BasicTokenizer(object):

    def __init__(self, do_lower_case=True, never_split=None):
        if never_split is None:
            never_split = []
        self.do_lower_case = do_lower_case
        self.never_split = never_split

    def tokenize(self, text, never_split=None):
        never_split = self.never_split + (never_split if never_split is not None else [])
        text = self._clean_text(text)
        orig_tokens = whitespace_tokenize(text)
        split_tokens = []
        for token in orig_tokens:
            if token not in never_split:
                if self.do_lower_case:
                    token = token.lower()
                token = self._run_strip_accents(token)
                split_tokens.extend(self._run_split_on_punc(token))
            else:
                split_tokens.append(token)

        output_tokens = whitespace_tokenize(" ".join(split_tokens))
        return output_tokens

    def _run_strip_accents(self, text):
        text = unicodedata.normalize("NFD", text)
        output = []
        for char in text:
            cat = unicodedata.category(char)
            if cat == "Mn":
                continue
            output.append(char)
        return "".join(output)

    def _run_split_on_punc(self, text, never_split=None):
        if never_split is not None and text in never_split:
            return [text]
        chars = list(text)
        i = 0
        start_new_word = True
        output = []
        while i < len(chars):
            char = chars[i]
            if _is_punctuation(char):
                output.append([char])
                start_new_word = True
            else:
                if start_new_word:
                    output.append([])
                start_new_word = False
                output[-1].append(char)
            i += 1

        return ["".join(x) for x in output]
    
    def _clean_text(self, text):
        output = []
        for char in text:
            cp = ord(char)
            if cp == 0 or cp == 0xfffd or _is_control(char):
                continue
            if _is_whitespace(char):
                output.append(" ")
            else:
                output.append(char)
        return "".join(output)
    
def _is_control(char):
    if char == "\t" or char == "\n" or char == "\r":
        return False
    cat = unicodedata.category(char)
    if cat.startswith("C"):
        return True
    return False

def _is_whitespace(char):
    if char == " " or char == "\t" or char == "\n" or char == "\r":
        return True
    cat = unicodedata.category(char)
    if cat == "Zs":
        return True
    return False

def _is_punctuation(char):
    cp = ord(char)
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
            (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False

In [38]:
from bert.tokenization import WordpieceTokenizer, load_vocab, convert_by_vocab

class FullTokenizer(object):
    def __init__(self, vocab_file, do_lower_case=True):
        self.vocab = load_vocab(vocab_file)
        self.inv_vocab = {v: k for k, v in self.vocab.items()}
        self.basic_tokenizer = BasicTokenizer(do_lower_case=do_lower_case, 
                                              never_split = ['[CLS]', '[MASK]', '[SEP]'])
        self.wordpiece_tokenizer = WordpieceTokenizer(vocab=self.vocab)

    def tokenize(self, text):
        split_tokens = []
        for token in self.basic_tokenizer.tokenize(text):
            for sub_token in self.wordpiece_tokenizer.tokenize(token):
                split_tokens.append(sub_token)

        return split_tokens

    def convert_tokens_to_ids(self, tokens):
        return convert_by_vocab(self.vocab, tokens)

    def convert_ids_to_tokens(self, ids):
        return convert_by_vocab(self.inv_vocab, ids)

In [41]:
tokenizer = FullTokenizer(vocab_file=BERT_VOCAB, do_lower_case=True)

In [42]:
text = '[CLS] დღე ძალიან კრგი დღეა [SEP]'
text_mask = text.replace('კრგი', '**mask**')
text_mask

'[CLS] დღეს მაგრად **mask** კოდებზე [SEP]'

In [43]:
def get_indices(mask, word):
    splitted = mask.split('**mask**')
    left = tokenizer.tokenize(splitted[0])
    middle = tokenizer.tokenize(word)
    right = tokenizer.tokenize(splitted[1])
    indices = [i for i in range(len(left))]
    for i in range(len(right)):
        indices.append(i + len(middle) + len(left))
    
    indices = indices[1:-1]
    tokenized = tokenizer.tokenize(mask.replace('**mask**',word))
    ids = tokenizer.convert_tokens_to_ids(tokenized)
    ids_left = tokenizer.convert_tokens_to_ids(left)
    ids_right = tokenizer.convert_tokens_to_ids(right)
    indices_word = ids_left + ids_right
    return ids, indices, indices_word[1:-1]

In [61]:
possible_states = corp.edit_candidates('კრგი')
possible_states

['კარგი', 'კრკი']

In [62]:
indices = [get_indices(text_mask, word) for word in possible_states]
ids, seq_ids, word_ids = list(zip(*indices))
ids

([101,
  1567,
  73685,
  1575,
  11577,
  44970,
  65363,
  1573,
  97383,
  33257,
  1573,
  47166,
  80536,
  102],
 [101,
  1567,
  73685,
  1575,
  11577,
  44970,
  65363,
  1573,
  20062,
  37183,
  1573,
  47166,
  80536,
  102])

In [75]:
bert_config = modeling.BertConfig.from_json_file(BERT_CONFIG)

AttributeError: module 'tensorflow' has no attribute 'gfile'

In [74]:
tf.__version__

'2.3.1'